<a href="https://colab.research.google.com/github/limkaram/Natural_language_processing_with_deep_learning/blob/main/Recurrent_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RNN(Recurrent_Neural_Network)

* model.add(SimpleRNN(hidden_size, input_shape=(timesteps, input_dim))) 
* hidden_size = 은닉 상태의 크기를 정의. 메모리 셀이 다음 시점의 메모리 셀과 출력층으로 보내는 값의 크기(output_dim)와도 동일
    - 중소형 모델의 경우 보통 128, 256, 512, 1024 등의 값을 가짐
* timesteps = 입력 시퀀스의 길이(input_length). 시점의 수
* input_dim = 입력의 크기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

In [ ]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(2, 10)))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


* RNN 층은 (batch_size, timesteps, input_dim) 크기의 3D 텐서를 입력으로 받음
* RNN 층은 사용자의 설정에 따라 두 가지 종류의 출력을 내보냄
    - 메모리 셀의 최종 시점의 은닉 상태만을 리턴하고자 한다면 (batch_size, output_dim) 크기의 2D 텐서를 리턴
    - 메모리 셀의 각 시점(time step)의 은닉 상태값들을 모아서 전체 시퀀스를 리턴하고자 한다면 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴
    - 이 차이는 인자인 return_sequences=True or False에 따라서 달라짐
    - 마지막 은닉 상태만 전달하도록 하면 many-to-one 문제를 풀 수 있고, 모든 시점의 은닉 상태를 전달하도록 하면, 다음층에 은닉층이 하나 더 있는 경우이거나 many-to-many 문제를 풀 수 있음
    

In [ ]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (8, 3)                    42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


* batch_size를 8로 하게되면, 출력의 크기가 (8, 3)이 됨. 즉, 2D 텐서가 반환됨

In [ ]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (8, 2, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


* batch_size를 8로 하게되면, 출력의 크기가 (8, 2, 3)인 3D 텐서가 됨

## numpy만으로 RNN 구현

In [10]:
import numpy as np

timesteps = 10  # 문장 토큰의 수
input_dim = 4  # 토큰을 표현한 벡터의 차원
hidden_size = 8  # 기억하려는 메모리 셀의 용량

inputs = np.random.random((timesteps, input_dim))
hidden_state_t = np.zeros((hidden_size, ))  # 최초 h(t) 0으로 초기화

In [12]:
Wx = np.random.random((hidden_size, input_dim))
Wh = np.random.random((hidden_size, hidden_size))
b = np.random.random((hidden_size, ))

In [25]:
total_hidden_states = []
for input in inputs:
    output_t = np.tanh(np.dot(Wx, input) + np.dot(Wh, hidden_state_t) + b)
    total_hidden_states.append(output_t)
    hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis=0)
print(total_hidden_states)
print(total_hidden_states.shape)

[[0.9974348  0.9997869  0.99999217 0.99988961 0.99997305 0.99999263
  0.99999011 0.99997781]
 [0.99655602 0.99948838 0.99998617 0.99976661 0.99996847 0.99998959
  0.99998342 0.9999474 ]
 [0.99783981 0.99981979 0.99999387 0.9998945  0.99997323 0.99999331
  0.99999374 0.99998231]
 [0.99856032 0.99940091 0.99998179 0.9998905  0.99998039 0.99999498
  0.99997187 0.99987017]
 [0.99863353 0.99964178 0.99998336 0.99990744 0.99998683 0.99999583
  0.99997711 0.99991424]
 [0.99845495 0.99975476 0.99998904 0.99992362 0.99998294 0.99999537
  0.99998425 0.99995563]
 [0.99892769 0.99961984 0.9999903  0.99993896 0.99997409 0.99999577
  0.99998412 0.99992973]
 [0.99889888 0.99983731 0.99999232 0.99991534 0.99998773 0.9999964
  0.99999418 0.99997148]
 [0.99833464 0.99984276 0.99999279 0.99991777 0.99998234 0.99999508
  0.9999924  0.99997852]
 [0.99820375 0.99986755 0.99999252 0.99988569 0.99998706 0.99999507
  0.99999453 0.99998224]]
(10, 8)


## keras API 활용 DRNN(Deep RNN) 구현

In [ ]:
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras import Sequential

model = Sequential()
model.add(SimpleRNN(hidden_size, return_sequences=True))
model.add(SimpleRNN(hidden_size, return_sequences=True))  # many-to-many 아키텍처의 경우

## BiRNN(Bidirectional RNN) 구현

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Bidirectional

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))

DBiRNN(Deep Bidirectional RNN) 구현

In [ ]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))